## Notebook to bulk download PlanetScope 4-band imagery through the Planet API

### _WARNING: has not been updated since the update from Planet SDK V1 -> V2_


Rainey Aberle

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and the Planet Labs Jupyter Notebooks for [API orders tutorials](https://github.com/planetlabs/notebooks/tree/665f165e59f2c3584976ad2dde569c649e631c0b/jupyter-notebooks/orders_api_tutorials).

### 1. Define paths in directory and setup filters for image search. 

#### _MODIFY THIS CELL_

In [ ]:
# -----Define paths in directory
site_name = 'SouthCascade'
# path to snow-cover-mapping/
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/' 
# path for saving image downloads
out_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/study-sites/' + site_name + '/imagery/RapidEye/'

# -----Area of Interest (AOI)
# Path and filename of your AOI shapefile
AOI_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/study-sites/' + site_name + '/AOIs/'
AOI_fn = 'SouthCascade_RGI_outline.shp'

# -----Date Range
# Format: 'YYYY-MM-DD'
start_date = '2011-05-01'
end_date = '2011-11-01'

# -----Cloud Cover Filter
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.8

# -----Item Type
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
# For PlanetScope Scene, item_type = "PSScene"
# For RapidEye Orthorectified Scene, item_type = "REOrthoTile"
item_type = "REOrthoTile"

# -----Asset Type
# Each Item Type has a number of image ("asset") types to choose from.
# See here for the asset types available for your item type:
# https://developers.planet.com/docs/apis/data/items-assets/
# For PlanetScope orthorectified 4-band surface reflectance products, asset_type = "ortho_analytic_4b_sr"
# For RapidEye orthorectified 5-band surface refelctance products, asset_type = "analytic_sr"
asset_type = "analytic_sr"

# -----AOI clipping
# Determine whether to clip images to the AOI (True/False)
# This greatly speeds up the ordering and downloading process and decreases the usage of your imagery quota
clip_to_AOI = True

# -----Sentinel-2 Harmonization
# option to harmonize PlanetScope imagery to Sentinel-2
harmonize = False # = True to harmonize

# -----Name of order
# Once requested, this makes it easier to locate the order on your "My Orders" page 
# in order to check the status or to re-download items via the website, etc. 
order_name = site_name + '_' + item_type[0:2] + '_' + start_date[0:4]
print('Order name: ' + request_name)

In [ ]:
# -----Import packages
import os
import glob
import json
from getpass import getpass
from planet import Auth, Session, OrdersClient, reporting
import requests
from requests.auth import HTTPBasicAuth
import geopandas as gpd
from shapely import geometry as sgeom
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# add path to functions
sys.path.insert(1, base_path+'functions/')
import PlanetScope_orders_utils as orders_utils
import pipeline_utils as f

### 2. Authenticate your Planet account

Requires your Planet API Key. To find your API Key, Login to your account at [planet.com](https://www.planet.com/) and go to "My Settings". 

If output is `<Repsonse [200]>`, authentication was successful. 

In [ ]:
# set API key as environment variable
API_key = getpass('Enter Planet API Key, then press Enter:')
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

### 3. Reformat AOI for querying

In [ ]:
# -----Read in the shapefile
AOI = gpd.read_file(AOI_path + AOI_fn)
# Reproject to WGS84 if necessary
AOI = AOI.to_crs('EPSG:4326')

# -----Convert AOI bounding box to geoJSON format
# Planet only excepts a bounding box as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {u'type': u'Polygon',
            u'coordinates': [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
            ]]
          }
# Convert AOI geojson to a rasterio Shape for Quick Search function
AOI_box_shape = sgeom.shape(AOI_box)

# -----Plot AOI and bounding box
fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
AOI.plot(ax=ax1, facecolor='none') # AOI
ax1.plot(*AOI_box_shape.exterior.xy) # AOI box
ax1.set_title('AOI outline and bounding box')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.show()

### 4. Compile filters to create a Quick Search request

Use Quick Search first to grab image IDs before ordering

In [ ]:
# -----Build QuickSearch request
QS_request = orders_utils.build_quick_search_request(AOI_box_shape, max_cloud_cover, start_date, end_date, item_type, asset_type)
        
# -----Planet API Quick Search using created request
# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)
# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
im_ids = sorted(im_ids)
print(len(im_ids),'images found')

In [ ]:
# only download images that don't already exist in directory
im_ids_filtered = []
for im_id in im_ids:
    num_exist = len(glob.glob(out_path + 'raw_images' + im_id+'*.tif'))
    if num_exist==0:
        im_ids_filtered = im_ids_filtered + [im_id]
print(str(len(im_ids_filtered)) + ' new images to be downloaded')

### 5. Place order and poll for success

This section compiles the request and places the order with outputs on the order status every ~5 seconds. Download will begin when the console outputs "success". 

In [ ]:
# -----Build new request
request = orders_utils.build_request_with_item_ids(request_name, AOI_box, clip_to_AOI, harmonize, 
                                                   im_ids_filtered, item_type, asset_type)

# -----Place order and download results
async def create_poll_and_download():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # create order via Orders client
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            # setting max_attempts=0 means there is no limit on the number of attempts
            await cl.wait(order['id'], callback=bar.update_state, max_attempts=0)

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'], directory=Path(out_path))
        
# remember: "await" to run the thing
await create_poll_and_download()

### _Optional:_ Plot downloaded images

In [ ]:
# -----Grab output image file names
# set image output folder
im_path = out_path + 'a590c954-2cd4-49a2-a149-1e808d645b0a/REOrthoTile/'
# change directory to im_path
os.chdir(im_path) 
# grab image file names
im_fns = glob.glob('*SR*.tif')

# -----Loop through files
for im_fn in im_fns:
    
    # open image
    im = rxr.open_rasterio(im_fn)
    # account for image scalar
    im_scalar = 1e4
    im = im / im_scalar
    # replace no data values with NaN
    im = im.where(im!=-9999)
    
    # plot RGB image
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.imshow(np.dstack([im.data[2], im.data[1], im.data[0]]), 
              extent=(im.x.data[0]/1e3, im.x.data[-1]/1e3, im.y.data[0]/1e3, im.y.data[-1]/1e3))
    ax.set_xlabel('Easting [km]')
    ax.set_ylabel('Northing [km]')
    ax.set_title(im_fn[0:8])
    plt.show()